<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [119]:
import pandas as pd
import psycopg2

In [ ]:
DBNAME =
USER =
PASSWORD =
HOST =
PORT =

#!!! LOGIN DETAILS REMOVED!!!

In [121]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

In [ ]:
# To avoid displaying warnings about using sqlalchemy
import warnings
warnings.filterwarnings("ignore")

# **Part 1. Preliminary Data Analysis** 

1. Execute an SQL query to **count the total number of job vacancies** in the database (`vacancies`).  

In [ ]:
query_1_1 = f'''select COUNT(id)
                from vacancies
            '''

In [ ]:
df = pd.read_sql_query(query_1_1, connection)
df

,count
0,49197


2. Determine the **number of registered employers** (`employers`).  

In [ ]:
query_1_2 = f'''select COUNT(id)
                from employers
            '''

In [ ]:
df = pd.read_sql_query(query_1_2, connection)
df

,count
0,23501


3. Calculate the **number of regions/cities** represented in the dataset (`areas`).  

In [ ]:
query_1_3 = f'''select COUNT(id)
                from areas
            '''

In [ ]:
df = pd.read_sql_query(query_1_3, connection)
df

,count
0,1362


4. Identify the **number of industries** covered in the database (`industries`).  

In [ ]:
query_1_4 = f'''select COUNT(id)
                from industries
            '''

In [ ]:
df = pd.read_sql_query(query_1_4, connection)
df

,count
0,294


***

### **Conclusions from Preliminary Data Analysis**  
The analysis of the database revealed the following insights:  
- The dataset contains **49,197 job vacancies** from **23,501 employers** across **1,362 regions**.  
- A total of **294 industries** are represented, indicating a broad coverage of business sectors.  
- Many companies are **direct competitors**, operating in the same geographic areas.  
- The average number of vacancies per employer is **~2.1**, suggesting that many organizations are hiring multiple specialists simultaneously.  

# **Part 2. Detailed Job Vacancy Analysis**  

1. Execute an SQL query to **determine the number of vacancies in each region (`area`)**.  
   - Sort the results in descending order. 

In [ ]:
query_2_1 = f'''select a.name area, COUNT(v.id) cnt
                from vacancies v
                left join areas a on v.area_id = a.id
                group by a.name
                order by 2 DESC 
            '''

In [ ]:
df = pd.read_sql_query(query_2_1, connection)
df

,area,cnt
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892
...,...,...
764,Тарко-Сале,1
765,Новоаннинский,1
766,Бирск,1
767,Сасово,1


2. Identify **the percentage of job postings that specify a salary** (at least one of the two salary fields).  

In [ ]:
query_2_2 = f'''select COUNT(id)
                from vacancies
                where (salary_from is not null) or (salary_to is not null)
            '''

In [ ]:
df = pd.read_sql_query(query_2_2, connection)
df

,count
0,24073


3. Calculate **the average lower and upper salary range** (rounded to whole numbers).  

In [ ]:
query_2_3 = f'''select round(AVG(salary_from)) avg_from, round(AVG(salary_to)) avg_to
                from vacancies
            '''

In [ ]:
df = pd.read_sql_query(query_2_3, connection)
df

,avg_from,avg_to
0,71065.0,110537.0


4. Retrieve **the number of vacancies for each combination of work schedule (`schedule`) and employment type (`employment`)**.  
   - Sort by descending order.

In [ ]:
query_2_4 = f'''select schedule, employment, count(id) cnt
                from vacancies
                group by 1,2
                order by 3 desc
            '''

In [ ]:
df = pd.read_sql_query(query_2_4, connection)
df

,schedule,employment,cnt
0,Полный день,Полная занятость,35367
1,Удаленная работа,Полная занятость,7802
2,Гибкий график,Полная занятость,1593
3,Удаленная работа,Частичная занятость,1312
4,Сменный график,Полная занятость,940
5,Полный день,Стажировка,569
6,Вахтовый метод,Полная занятость,367
7,Полный день,Частичная занятость,347
8,Гибкий график,Частичная занятость,312
9,Полный день,Проектная работа,141


5. Determine **which experience levels (`experience`) are least frequently mentioned in job postings**.  

In [ ]:
query_2_5 = f'''select experience as "Требуемый опыт работы", count(id) cnt
                from vacancies
                group by 1
                order by 2
            '''

In [ ]:
df = pd.read_sql_query(query_2_5, connection)
df

,Требуемый опыт работы,cnt
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


***

### **Conclusions from Detailed Job Vacancy Analysis**  

🔹 **Job distribution by region:**  
The largest cities, such as **Moscow, Saint Petersburg, and Novosibirsk**, have the highest number of job openings, aligning with general labor market trends.  

🔹 **Salary information availability:**  
- **49% of job postings** include at least one salary field.  
- A significant portion of employers **do not disclose salary ranges**, making automated analysis and prediction more challenging.  

🔹 **Work schedule and employment type:**  
- Most employers offer **full-time, in-office positions**.  
- Flexible schedules and remote work options are available but less common.  

🔹 **Salary expectations:**  
- The average salary range is **from 71,000 to 110,000 RUB**.  
- The most in-demand employees have **1–3 years of experience**. 


# **Part 3. Employer Analysis** 

1. Execute an SQL query to **identify the TOP-5 employers** by the number of job vacancies.  

In [ ]:
query_3_1 = f'''select e.name, count(v.id) cnt
                from vacancies v
                left join employers e on v.employer_id=e.id
                group by 1
                order by 2 desc
                limit 5
            '''

In [ ]:
df = pd.read_sql_query(query_3_1, connection)
df

,name,cnt
0,Яндекс,1933
1,Ростелеком,491
2,Тинькофф,444
3,СБЕР,428
4,Газпром нефть,331


2. Determine the **number of employers and job vacancies in each region**.  
   - Identify the region with the highest number of employers but no vacancies.  

In [ ]:
query_3_2 = f'''select a.name, count(distinct e.id) employers, count(distinct v.id) vacancies
                from areas a 
                left join employers e on a.id=e.area
                left join vacancies v on v.area_id=e.area
                where v.id is null
                group by 1
                order by 3, 2 desc
            '''

In [ ]:
df = pd.read_sql_query(query_3_2, connection)
df

,name,employers,vacancies
0,Россия,410,0
1,Казахстан,207,0
2,Московская область,75,0
3,Краснодарский край,19,0
4,Беларусь,18,0
...,...,...,...
889,Жабинка,0,0
890,Железногорск-Илимский,0,0
891,Жетысай,0,0
892,Жигалово,0,0


3. Calculate the **number of regions where each employer has job postings**.  
   - Sort the results in descending order.

In [ ]:
query_3_3 = f'''select e.name employer, count(distinct v.area_id) cnt
                from vacancies v
                left join employers e on v.employer_id=e.id
                group by 1
                order by 2 desc
            '''

In [ ]:
df = pd.read_sql_query(query_3_3, connection)
df

,employer,cnt
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71
...,...,...
14761,UniSol,1
14762,UNISTORY LLC,1
14763,UNIT6,1
14764,United Distribution,1


4. Count **the number of employers without an assigned industry**.  

In [ ]:
query_3_4 = f'''select count(distinct e.id)
                from employers e
                left join employers_industries s on e.id=s.employer_id
                where s.industry_id is null
            '''

In [ ]:
df = pd.read_sql_query(query_3_4, connection)
df

,count
0,8419


5. Find **the company ranked third alphabetically** among employers with four industry classifications.  

In [ ]:
query_3_5 = f'''select e.name, count(s.industry_id)
                from employers e
                left join employers_industries s on e.id=s.employer_id
                group by 1
                having count(s.industry_id)=4
                order by 1
                offset 2
                limit 1
            '''

In [ ]:
df = pd.read_sql_query(query_3_5, connection)
df

,name,count
0,2ГИС,4


6. Identify **the number of employers specializing in software development**.  

In [ ]:
query_3_6 = f'''select count(distinct s.employer_id)
                from employers_industries s
                left join industries i on i.id=s.industry_id
                where i.name = 'Разработка программного обеспечения' 
            '''

In [ ]:
df = pd.read_sql_query(query_3_6, connection)
df

,count
0,3553


7. Analyze **Yandex job postings** in cities with over 1 million residents.  
   - Include a `Total` row with the overall number of job openings.

In [ ]:
import urllib.parse as req

path_common = req.urlparse('https://ru.wikipedia.org/wiki/Города-миллионеры_России#Список_городов-миллионеров')
url = path_common.scheme + '://' + req.quote(path_common.netloc) +  req.quote(path_common.path) + '#'+ req.quote(path_common.fragment)

cities=pd.read_html(url)[0]
cities=tuple(cities['Город'])
display(cities)

('Москва',
 'Санкт-Петербург',
 'Новосибирск',
 'Екатеринбург',
 'Казань',
 'Красноярск',
 'Нижний Новгород',
 'Челябинск',
 'Уфа',
 'Самара',
 'Ростов-на-Дону',
 'Краснодар',
 'Омск',
 'Воронеж',
 'Пермь',
 'Волгоград')

In [ ]:
query_3_7 = f'''select a.name, count(v.id) cnt
                from vacancies v
                left join areas a on v.area_id=a.id
                left join employers e on v.employer_id=e.id
                where e.name = 'Яндекс' and a.name in {cities}
                group by a.name
                
                union
                select 'Total', count(v.id)
                from vacancies v
                left join areas a on v.area_id=a.id
                left join employers e on v.employer_id=e.id
                where e.name = 'Яндекс' and a.name in {cities}
                order by 2
                
            '''

In [ ]:
df = pd.read_sql_query(query_3_7, connection)
df

,name,cnt
0,Омск,21
1,Челябинск,22
2,Красноярск,23
3,Волгоград,24
4,Пермь,25
5,Казань,25
6,Ростов-на-Дону,25
7,Уфа,26
8,Самара,26
9,Краснодар,30


***

### **Conclusions from Employer Analysis**  

🔹 **Top Employers:**  
The leading employers in terms of job postings include **Яндекс, Ростелеком, Тинькофф**.  
These organizations are either **major IT firms** or companies that heavily rely on data processing, creating a strong demand for tech professionals.  

🔹 **Software Development Sector:**  
- **3,553 out of 23,501 employers** in the dataset are focused on **software development**.  
- This highlights **significant employment opportunities in the IT industry**.  

🔹 **Geographic Reach:**  
- **Yandex** is the **leader in geographic coverage**, with job postings in most **major metropolitan areas**.  
- This indicates its continued expansion and a high demand for specialists nationwide.  

# **Part 4. Domain-Specific Analysis**  

1. Determine the **number of data-related job vacancies**.  
   - A job is classified as "data-related" if its title contains `'data'` or `'данн'`. 

In [ ]:
query_4_1 = f'''select count(id)
                from vacancies
                where lower(name) like '%data%' or lower(name) like '%данн%'
            '''

In [ ]:
df = pd.read_sql_query(query_4_1, connection)
df

,count
0,1771


2. Calculate the **number of entry-level Data Scientist job postings**.  
   - Included job titles:  
     * 'data scientist'  
     * 'data science'  
     * 'исследователь данных'  
     * 'ML' (excluding `HTML`)  
     * 'machine learning'  
     * 'машинн%обучен%'  
   - A **Junior-level** position meets at least one of these criteria:  
     * The title contains `'junior'`.  
     * Required experience = `'Нет опыта'`.  
     * Employment type = `'Стажировка'`.  
 

In [ ]:
query_4_2 = f'''select count(distinct id)
                from vacancies
                where ((name ilike '%data scientist%'
                or name ilike '%data science%' 
                or name ilike '%исследователь данных%'
                or name ilike '%machine learning%'
                or name ilike '%машинн%обучен%')
                or (name not like '%HTML%' and name like '%ML%'))
                and (name ilike '%junior%' 
                or experience ilike '%нет опыта%' 
                or employment ilike '%стажировка%')
            '''

In [ ]:
df = pd.read_sql_query(query_4_2, connection)
df

,count
0,51


3. Count **how many DS job postings require SQL or PostgreSQL as a key skill**.  

In [ ]:
query_4_3 = f'''select count(distinct id)
                from vacancies
                where ((name ilike '%data scientist%'
                or name ilike '%data science%' 
                or name ilike '%исследователь данных%'
                or name ilike '%machine learning%'
                or name ilike '%машинн%обучен%')
                or (name not like '%HTML%' and name like '%ML%'))
                and (key_skills ilike '%SQL%'
                or key_skills ilike '%postgres%')
            '''

In [ ]:
df = pd.read_sql_query(query_4_3, connection)
df

,count
0,201


4. Assess **the popularity of Python** among employer requirements.  
   - Number of job postings where Python is listed as a key skill. 

In [ ]:
query_4_4 = f'''select count(distinct id)
                from vacancies
                where ((name ilike '%data scientist%'
                or name ilike '%data science%' 
                or name ilike '%исследователь данных%'
                or name ilike '%machine learning%'
                or name ilike '%машинн%обучен%')
                or (name not like '%HTML%' and name like '%ML%'))
                and (key_skills ilike '%Python%')
            '''

In [ ]:
df = pd.read_sql_query(query_4_4, connection)
df

,count
0,351


5. Determine **the average number of key skills** listed in DS job postings.  
   - Round the result to two decimal places.

In [ ]:
query_4_5 = f'''select round(avg(length(key_skills)-length(replace(key_skills, CHR(9), ''))+1), 2)
                from vacancies
                where ((name ilike '%data scientist%'
                or name ilike '%data science%' 
                or name ilike '%исследователь данных%'
                or name ilike '%machine learning%'
                or name ilike '%машинн%обучен%')
                or (name not like '%HTML%' and name like '%ML%'))
                and key_skills is not null
                order by 1
            '''

In [ ]:
df = pd.read_sql_query(query_4_5, connection)
df

,round
0,6.41


6. Calculate **the average salary for DS jobs** based on required experience levels.  
   - Identify **the expected salary range for candidates with 3–6 years of experience**.  
   - Round to the nearest whole number. 

In [ ]:
query_4_6 = f'''select experience, round(avg((coalesce(salary_from, salary_to)+coalesce(salary_to, salary_from))/2), 0) as avg_salary
                from vacancies
                where ((name ilike '%data scientist%'
                or name ilike '%data science%' 
                or name ilike '%исследователь данных%'
                or name ilike '%machine learning%'
                or name ilike '%машинн%обучен%')
                or (name not like '%HTML%' and name like '%ML%'))
                and (salary_from is not null or salary_to is not null)
                group by experience   
            '''

In [ ]:
df = pd.read_sql_query(query_4_6, connection)
df

,experience,avg_salary
0,Нет опыта,74643.0
1,От 1 года до 3 лет,139675.0
2,От 3 до 6 лет,243115.0


### **Conclusions from Domain-Specific Analysis**  

🔹 **Filtered Job Selection:**  
After applying DS-specific criteria, the number of relevant vacancies **was reduced to 1,771**.  

🔹 **Job Availability for Entry-Level Candidates:**  
- **Only 2.8% of DS job postings** are open to **candidates without experience**.  
- This confirms the **high entry barrier** for Data Science positions.  

🔹 **Skill Popularity:**  
- **20% of DS job postings** require **Python** as a key skill.  
- **11% of DS job postings** require **SQL**.  
- On average, employers look for candidates **with 6–7 key skills**.  

🔹 **Salary Expectations:**  
- Salaries in Data Science **increase rapidly with experience**.  
- Experienced specialists earn **3–4 times more** than entry-level employees.  
- The average salary for candidates with **3–6 years of experience** is **243 115 RUB**. 

***

# **Part 5. Additional Research**

🔹 **Choosing the Best Region for Job Seekers** 

In [ ]:
# The first query selects regions where job vacancies for our clients are currently open  
# and lists them in descending order by the number of potential employers.  
query_5_1 = f'''select a.name as "название региона", COUNT(distinct v.employer_id) as "с открытыми вакансиями"
                from areas a
                left join vacancies v on a.id=v.area_id
                where ((v.name ilike '%data scientist%'
                or v.name ilike '%data science%' 
                or v.name ilike '%исследователь данных%'
                or v.name ilike '%machine learning%'
                or v.name ilike '%машинн%обучен%')
                or (v.name not like '%HTML%' and v.name like '%ML%'))
                group by 1
                order by 2 desc
            '''

In [ ]:
# Based on the query results, we form the first part of the DataFrame  
# and create a tuple with the names of the selected regions.
df1 = pd.read_sql_query(query_5_1, connection)
regions = tuple(df1['название региона'])

In [ ]:
# The second query retrieves the number of potential employers for IT specialists  
# in the regions selected in the first query.  
# This query requires joining four tables, even though the final output only contains data from two of them.  
# The other tables are used for linking and filtering conditions.  
query_5_2 = f'''select a.name as "название региона", COUNT(distinct e.id) as "работодатели в ИТ сфере"
                from areas a
                left join employers e on a.id=e.area
                left join employers_industries s on e.id=s.employer_id
                left join industries i on s.industry_id=i.id
                where (i.name like '%ИТ%'
                or i.name ilike '%интернет%' 
                or i.name like 'Разработка программного обеспечения'
                or i.name like 'Мобильная связь'
                or i.name like 'Банк')
                and a.name in {regions}
                group by 1
                order by 2 desc
            '''

In [ ]:
# Forming the second part of the table.  
df2 = pd.read_sql_query(query_5_2, connection)

# Merging the data into a single table.  
merged_df=df1.merge(
    df2,
    how='left',
    on='название региона'
)
# Replacing missing values with 0 and displaying the final result. 
merged_df=merged_df.fillna(0)
merged_df

,название региона,с открытыми вакансиями,работодатели в ИТ сфере
0,Москва,130,1739.0
1,Санкт-Петербург,47,678.0
2,Новосибирск,17,166.0
3,Казань,13,133.0
4,Алматы,11,262.0
5,Нижний Новгород,9,95.0
6,Минск,9,515.0
7,Екатеринбург,6,134.0
8,Томск,6,57.0
9,Краснодар,6,81.0


- To determine **the most promising regions**, it is important to consider **not only the number of current vacancies but also the total number of employers**.  
- As expected, the **highest concentration of vacancies and employers** is in **Moscow**.  
- However, **many companies register their headquarters in Moscow**, which **may distort the real distribution**.  
- Beyond Moscow, **capital cities and metropolitan areas** stand out as **attractive options for relocation and career growth**.

🔹 **Optimal Work Schedule for Data Scientists**

In [ ]:
query_5_3 = f'''select schedule, employment, round(AVG(salary_from)) avg_from, round(AVG(salary_to)) avg_to, count(id) cnt
                from vacancies
                where ((name ilike '%data scientist%'
                or name ilike '%data science%' 
                or name ilike '%исследователь данных%'
                or name ilike '%machine learning%'
                or name ilike '%машинн%обучен%')
                or (name not like '%HTML%' and name like '%ML%'))
                group by 1,2
                order by 3 desc, 4 desc
            '''

In [ ]:
df = pd.read_sql_query(query_5_3, connection)
df

,schedule,employment,avg_from,avg_to,cnt
0,Гибкий график,Частичная занятость,NaN,NaN,4
1,Полный день,Стажировка,NaN,NaN,2
2,Удаленная работа,Проектная работа,NaN,90000.0,2
3,Гибкий график,Стажировка,NaN,37000.0,2
4,Удаленная работа,Полная занятость,176850.0,242138.0,110
5,Полный день,Полная занятость,140763.0,228936.0,319
6,Гибкий график,Полная занятость,140000.0,200000.0,37
7,Удаленная работа,Частичная занятость,119000.0,NaN,3
8,Полный день,Частичная занятость,45000.0,100000.0,1


- Based on salary analysis and vacancy numbers, **remote work with full-time employment** is the **most financially beneficial option**.  
- This format offers **a sufficient number of job opportunities**, and salaries are **on average 13% higher** than those for traditional office-based roles.  
- The first four schedule and employment type combinations **are less attractive** due to **a limited number of job postings and unclear salary ranges**.  

In [ ]:
# Closing the database connection. 
connection.close()

---

## **General Conclusions from the Project**  

📌 **Demand for Data Science Specialists**  
- Out of **49,197 job postings**, only **1,771 (3.6%)** are related to **Data Science**.  
- This confirms that **data professionals are in high demand**, but the job market remains **competitive**.  
- The largest hiring hubs are **capital cities and major metropolitan areas**.  

📌 **Key Factors in Job Matching**  
- **Work experience** and **essential skills** (Python, SQL, ML) are **the primary hiring criteria**.  
- **Remote work is gaining popularity**, offering **higher average salaries**.  
- **Matching candidates with employer requirements** is crucial for job search success.  

📌 **Recommendations for Building an ML-Based Job Matching System**  
- **A single data source (HeadHunter) is insufficient** → integrating multiple job boards would improve **recommendation accuracy**.  
- **Additional factors** (job categories, market trends, skill requirements) can **enhance the model’s predictive power**.  
- **Long-term trend analysis** (e.g., growth in remote work) will ensure **the model remains relevant over time**.  